In [48]:
import pandas as pd
from pathlib import Path
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define the base and data directories
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR.parents[0] / "transfer_data"
CLUBS_FILE = "clubs.csv"
ELOS_FILE = "club_elos.csv"

# Load CSV data
clubs_df = pd.read_csv(os.path.join(DATA_DIR, CLUBS_FILE), sep=",", encoding="UTF-8")
elos_df = pd.read_csv(os.path.join(DATA_DIR, ELOS_FILE), sep=",", encoding="UTF-8")
elos_df = elos_df.head(632)  # Restrict the ELO data to the first 632 rows


# Define the fuzzy merge function
def fuzzy_merge(
    df_1, df_2, key1, key2, threshold=90, limit=2, scorer=fuzz.partial_ratio
):
    """
    Fuzzy merge two dataframes based on a similarity score.
    """
    s = df_2[key2].tolist()
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit, scorer=scorer))
    df_1["matches"] = m
    df_1["matches"] = df_1["matches"].apply(
        lambda x: ", ".join([i[0] for i in x if i[1] >= threshold])
    )

    df_1 = df_1.set_index(list(df_1)[0])
    df_1 = df_1.reset_index(drop=True)
    return df_1

# Initialize the unmatched dataframe
unmatched_df = elos_df.copy()
unmatched_df.columns = unmatched_df.columns.str.lower()
unmatched_df = unmatched_df[unmatched_df['country'] == 'ENG']

# Replace 'Man' with 'Manchester' in the 'club' column (case-insensitive)
unmatched_df['club'] = unmatched_df['club'].str.replace(r'\bMan\b', 'Manchester', case=False, regex=True)

# Filter clubs_df so that it only contains English teams
clubs_df = clubs_df[clubs_df['domestic_competition_id'] == 'GB1']

# Start the threshold at a higher value and keep lowering it until all teams are matched
initial_threshold = 85
threshold_step = 5  # Decrease the threshold by 5 in each iteration
min_threshold = 60  # Stop decreasing if threshold reaches this minimum
all_results = []
counter = 0

while not unmatched_df.empty and initial_threshold >= min_threshold:
    # Perform fuzzy merge with the current threshold
    matched_df = fuzzy_merge(
        unmatched_df.copy(),
        clubs_df,
        "club",
        "name",
        threshold=initial_threshold,
        limit=1,
        scorer=fuzz.partial_ratio,
    )

    # Save the intermediate results for each iteration
    matched_df.to_csv(f"match_V{counter}.csv", index=True)
    counter += 1

    # Filter out matched rows
    matched_clubs = matched_df[matched_df["matches"] != ""]
    matched_names = matched_clubs["matches"].str.split(", ").explode().unique()

    # Remove matched clubs from clubs_df and unmatched_df
    clubs_df = clubs_df[~clubs_df["name"].isin(matched_names)]
    unmatched_df = unmatched_df[~unmatched_df["club"].isin(matched_clubs["club"])]

    # Append current iteration's matched results to all_results
    all_results.append(matched_clubs)

    # If there are unmatched clubs, reduce the threshold
    if not unmatched_df.empty:
        print(f"Unmatched teams remaining: {len(unmatched_df)}")
        initial_threshold -= threshold_step

# Combine all matched results and save
final_result = pd.concat(all_results, ignore_index=True)
final_result.to_csv("final_match.csv", index=False)

# Save any unmatched entries
if not unmatched_df.empty:
    unmatched_df.to_csv("unmatched.csv", index=False)
    final = pd.concat([final_result, unmatched_df], ignore_index=True)
    # Drop last column cuz it shouldn't be there
    final = final[final.columns.intersection(final_result.columns)]
    final.to_csv('final_match_result.csv', index=False)
else:
    print("All teams were matched!")


Unmatched teams remaining: 11
Unmatched teams remaining: 10
Unmatched teams remaining: 10
Unmatched teams remaining: 10
Unmatched teams remaining: 9
Unmatched teams remaining: 7
